In [15]:
# Impordid
from src.transformers.models.bert.tokenization_bert import BertTokenizer
from src.transformers.models.bert.modeling_bert import BertEmbeddings, BertModel, BertForMaskedLM
import torch
from estnltk import Text
from estnltk.corpus_processing.parse_enc import parse_enc_file_iterator
from src.transformers.models.bert.configuration_bert import BertConfig
from transformers import AdamW
from tqdm.auto import tqdm
from transformers import pipeline
import numpy as np

In [16]:
# Tokeniseerija
tokenizer = BertTokenizer(vocab_file = "vocab_final.txt", vocab_file_form = "vocab_form.txt")

In [17]:
%%time
# Korpus
# Loeme tekstid sisse, laseme estnltk-l laused leida ning moodustame lausetest treening- ja testhulg
# https://github.com/estnltk/estnltk/blob/main/tutorials/corpus_processing/importing_text_objects_from_corpora.ipynb
input_file = "korpus/estonian_nc17.vert.01"
n = 2 # Mitu teksti korpusesse lugeda
korpus = []
l = 0
for text_obj in parse_enc_file_iterator(input_file):
    korpus.append(text_obj)
    if l > n:
        break
    l += 1


train_laused = []
for tekst in train:
    for span in tekst.original_sentences:
        train_laused.append(tekst.text[span.start:span.end])
        
test_laused = []
for tekst in test:
    for span in tekst.original_sentences:
        test_laused.append(tekst.text[span.start:span.end])

CPU times: total: 62.5 ms
Wall time: 50 ms


In [18]:
# Dataseti loomine dataloaderi jaoks
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return self.encodings["input_ids"].shape[0]
    def __getitem__(self, i):
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [19]:
# Maskimine ja labelid
def mlm(tensor):
    labels = tensor.detach().clone()
    rand = torch.rand(tensor[:, :, 0].shape)
    mask_arr = (rand < 0.15) * (tensor[:, :, 0] > 5)
    for i in range(tensor[:, :, 0].shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        selection_masked = torch.where(mask_arr[i] == 0)[0].tolist()
        tensor[i, selection] = 4 # [MASK] tokeni ID mõlemas vocabis
        labels[i, selection_masked] = -100 # Et mudel arvutaks lossi ainult masked tokenite pealt
        
    return tensor, labels

In [27]:
# Andmestiku ettevalmistamine treenimiseks või mudelis kasutamiseks
def prepare_data(data):
    input_ids = []
    mask = []
    labels_vorm = []
    labels_lemma = []
    
    sample = tokenizer(data, max_length = 16, padding = "max_length", truncation = True, return_tensors = "pt")
    new_ids, new_labels = mlm(sample.input_ids.detach().clone())
    input_ids.append(new_ids)
    mask.append(sample.attention_mask)
    labels_vorm.append(new_labels[:, :, 1])
    labels_lemma.append(new_labels[:, :, 0])
    
    input_ids = torch.cat(input_ids)
    mask = torch.cat(mask)
    labels_vorm = torch.cat(labels_vorm)
    labels_lemma = torch.cat(labels_lemma)
    
    encodings = {
    "input_ids" : input_ids,
    "attention_mask" : mask,
    "labels_vorm" : labels_vorm,
    "labels_lemma" : labels_lemma,
    }
    
    dataset = Dataset(encodings)
    return dataset


# Dataloaderi loomine mudeli treenimiseks
dataloader_train = torch.utils.data.DataLoader(prepare_data(train_laused), batch_size = 16, shuffle = True)

In [30]:
%%time
# Mudeli treenimine

config = BertConfig(
    vocab_size = tokenizer.vocab_size,
    vocab_size_form = tokenizer.vocab_size_form,
    tie_word_embeddings = False
)

model = BertForMaskedLM(config)
linear = torch.nn.Linear(tokenizer.vocab_size, tokenizer.vocab_size_form)
CE = torch.nn.CrossEntropyLoss()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)
optim = AdamW(model.parameters(), lr = 1e-4)
epochs = 5

for epoch in range(epochs):
    loop = tqdm(dataloader_train, leave = True)
    for batch in loop:
        optim.zero_grad()
        input_ids = batch["input_ids"]
        mask = batch["attention_mask"]
        labels_vorm = batch["labels_vorm"]
        labels_lemma = batch["labels_lemma"]
        output_lemma = model(input_ids, attention_mask = mask, labels = labels_lemma)
        loss_lemma = CE(output_lemma.logits.view(-1, tokenizer.vocab_size), labels_lemma.view(-1))
        output_vorm = linear(output_lemma.logits)
        loss_vorm = CE(output_vorm.view(-1, tokenizer.vocab_size_form), labels_vorm.view(-1))
        loss = loss_lemma + loss_vorm
        loss.backward()
        optim.step()
        loop.set_description(f"Epoch: {epoch}")
        loop.set_postfix(loss = loss.item())

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

CPU times: total: 3min 17s
Wall time: 26.3 s


In [35]:
output_lemma.logits.size()
labels_lemma.view(-1)

tensor([ -100,  -100,  -100,   738,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,     9,  -100,  -100,
         -100,  -100,   100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100, 14244,  -100,
         2674,  -100,   343,  -100,  -100,  -100,  -100,  -100,    32,  -100,
         -100,  -100,  -100,  -100,   693,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,    21,  -100,  -100,  -100,  -100,  -100, 

In [23]:
# Testimine
dataloader_test = torch.utils.data.DataLoader(prepare_data(test_laused), batch_size = 16)
kokku_labelid_vorm = np.array([-1])
kokku_labelid_lemma = np.array([-1])
kokku_ennustused_vorm = np.array([-1])
kokku_ennustused_lemma = np.array([-1])
#loop2 = tqdm(dataloader_test, leave = True) # Testandmed
loop2 = tqdm(dataloader_train, leave = True) # Treeningandmetel võiks accuracy tulla 1 kui epocheid piisavalt
for batch in loop2:
    input_ids = batch["input_ids"]
    labels_vorm = batch["labels_vorm"]
    labels_lemma = batch["labels_lemma"]
    
    output_lemma = model(input_ids).logits
    output_vorm = linear(output_lemma)
    indeksid = np.where(input_ids[:, :, 0].flatten() == 4)
    labels_vorm = labels_vorm.flatten()[indeksid].detach().numpy()
    labels_lemma = labels_lemma.flatten()[indeksid].detach().numpy()
    ennustused_vorm = np.argmax(output_vorm.flatten(end_dim = 1)[indeksid].detach().numpy(), axis = 1)
    ennustused_lemma = np.argmax(output_lemma.flatten(end_dim = 1)[indeksid].detach().numpy(), axis = 1)
    
    kokku_labelid_vorm = np.concatenate((kokku_labelid_vorm, labels_vorm))
    kokku_labelid_lemma = np.concatenate((kokku_labelid_lemma, labels_lemma))
    
    kokku_ennustused_vorm = np.concatenate((kokku_ennustused_vorm, ennustused_vorm))
    kokku_ennustused_lemma = np.concatenate((kokku_ennustused_lemma, ennustused_lemma))

  0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
kokku_labelid_vorm
kokku_labelid_lemma

array([   -1,  2301, 36236,     6,   343, 14244, 14244,    10,  3277,
         343, 14244, 20631,   493,  2624,   426,  4119, 36246,    24,
          58,    14,    35,   303,    24,   154,   142,    39,     7,
         603,  4897, 14240,     8,     8,    29,    10,  1642,   764,
         937,   674,  3798, 19150,  1642,   905,  4117,  3202,  2012,
          14,    28,   227,   590,     6,   998,     7,    40,  2903,
           7,   336,   823,   686,  1082,  3276,  2256,   408,  1843,
        1646,   133, 36248,  2352,   546, 36250,  4481,     6,  2301,
           6,    82,  1203], dtype=int64)

In [25]:
kokku_ennustused_vorm
kokku_ennustused_lemma

array([   -1, 14244, 14244, 14244, 14244, 14244, 14244, 14244, 14244,
       14244, 14244, 14244, 14244, 14244, 14244, 14244, 14244, 14244,
       14244, 14244, 14244, 14244, 14244, 14244, 14244, 14244, 14244,
       14244, 14244, 14244, 14244, 14244, 14244, 14244, 14244, 14244,
       14244, 14244, 14244, 14244, 14244, 14244, 14244, 14244, 14244,
       14244, 14244, 14244, 14244, 14244, 14244, 14244, 14244, 14244,
       14244, 14244, 14244, 14244, 14244, 14244, 14244, 14244, 14244,
       14244, 14244, 14244, 14244, 14244, 14244, 14244, 14244, 14244,
       14244, 14244, 14244], dtype=int64)

In [26]:
sum(kokku_ennustused_vorm[1:] == kokku_labelid_vorm[1:])/len(kokku_labelid_vorm[1:])
sum(kokku_ennustused_lemma[1:] == kokku_labelid_lemma[1:])/len(kokku_labelid_lemma[1:])

0.04054054054054054

In [13]:
#torch.save(model, "mudel.pth")